<a href="https://colab.research.google.com/github/Alvin-Buana/Tourist-Recommendation/blob/main/Content_Based_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/aprabowo/indonesia-tourism-destination")

100%|██████████| 158k/158k [00:00<00:00, 18.6MB/s]

In [3]:
import pandas as pd
import numpy as np

In [4]:
data_rating = pd.read_csv('/content/indonesia-tourism-destination/tourism_rating.csv')
data_tourism = pd.read_csv('indonesia-tourism-destination/tourism_with_id.csv')


In [10]:
category=[]
place_tourism = data_tourism.copy()
place_tourism = place_tourism[['Place_Name','Category']]
category = list(data_tourism.Category.unique())


for x in category:
    place_tourism[x] = 0
for i in range(len(place_tourism.Category)):
        x = place_tourism.Category[i]
        place_tourism[x][i]=1
place_tourism = place_tourism.set_index('Place_Name')
place_tourism

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Category,Budaya,Taman Hiburan,Cagar Alam,Bahari,Pusat Perbelanjaan,Tempat Ibadah
Place_Name,,,,,,,
Monumen Nasional,Budaya,1,0,0,0,0,0
Kota Tua,Budaya,1,0,0,0,0,0
Dunia Fantasi,Taman Hiburan,0,1,0,0,0,0
Taman Mini Indonesia Indah (TMII),Taman Hiburan,0,1,0,0,0,0
Atlantis Water Adventure,Taman Hiburan,0,1,0,0,0,0
...,...,...,...,...,...,...,...
Museum Mpu Tantular,Budaya,1,0,0,0,0,0
Taman Bungkul,Taman Hiburan,0,1,0,0,0,0
Taman Air Mancur Menari Kenjeran,Taman Hiburan,0,1,0,0,0,0


In [11]:
place_tourism = place_tourism.drop('Category',axis=1)
place_tourism = csr_matrix(place_tourism.values)

# Modelling

## Set the model function

In [17]:
def model(model,place):
  model_df = model(place_tourism)
  model_df = pd.DataFrame(model_df, index=data_tourism['Place_Name'], columns=data_tourism['Place_Name'])
  count = 5
  items=data_tourism[['Place_Name','Category','City','Rating','Price','Time_Minutes']]
  index = model_df.loc[:,place].to_numpy().argpartition(range(-1, -count, -1))
  closest = model_df.columns[index[-1:-(count+2):-1]]
  closest = closest.drop(place, errors='ignore')
  new_rec = pd.DataFrame(closest).merge(items).head(count)
  cat = data_tourism.Category[data_tourism['Place_Name']== place].iloc[0]
  where = data_tourism.City[data_tourism['Place_Name']== place].iloc[0]
  new_rec = new_rec.style.set_caption("Recommendation for {} (Category : {} ,Place : {})".format(place,cat,where))

  return new_rec


## Using Sigmoid Kernel

In [18]:
from sklearn.metrics.pairwise import sigmoid_kernel

place = data_tourism.Place_Name.sample(1).iloc[0]

test = model(sigmoid_kernel,place)
test

,Place_Name,Category,City,Rating,Price,Time_Minutes
0,Pantai Pok Tunggal,Bahari,Yogyakarta,4.500000,10000,nan
1,Pantai Drini,Bahari,Yogyakarta,4.500000,10000,nan
2,Pantai Sadranan,Bahari,Yogyakarta,4.600000,10000,nan
3,Pantai Samas,Bahari,Yogyakarta,4.000000,4000,45.000000
4,Pantai Nguluran,Bahari,Yogyakarta,4.100000,10000,nan


## Using Cosine Similarity

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
place = data_tourism.Place_Name.sample(1).iloc[0]
test = model(cosine_similarity,place)
test

,Place_Name,Category,City,Rating,Price,Time_Minutes
0,Dago Dreampark,Taman Hiburan,Bandung,4.200000,40000,150.000000
1,Panghegar Waterboom Bandung,Taman Hiburan,Bandung,4.300000,75000,nan
2,Alive Museum Ancol,Taman Hiburan,Jakarta,4.300000,200000,nan
3,Glamping Lakeside Rancabali,Taman Hiburan,Bandung,4.400000,30000,nan
4,Taman Miniatur Kereta Api,Taman Hiburan,Bandung,4.400000,15000,nan


## Using Eucledian Distance

In [21]:
from sklearn.metrics.pairwise import euclidean_distances
place = data_tourism.Place_Name.sample(1).iloc[0]
test = model(euclidean_distances,place)
test

,Place_Name,Category,City,Rating,Price,Time_Minutes
0,Gereja Perawan Maria Tak Berdosa Surabaya,Tempat Ibadah,Surabaya,4.800000,10000,nan
1,Bunker Kaliadem Merapi,Cagar Alam,Yogyakarta,4.500000,3000,45.000000
2,Goa Jomblang,Cagar Alam,Yogyakarta,4.600000,500000,nan
3,Ledok Sambi,Cagar Alam,Yogyakarta,4.600000,0,150.000000
4,Seribu Batu Songgo Langit,Cagar Alam,Yogyakarta,4.600000,2000,90.000000
